In [ ]:
#https://www.kaggle.com/justjun0321/job-recommendation-find-you-job-at-google


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv('job_skills.csv')

In [ ]:
df.head(3)

,Company,Title,Category,Location,Responsibilities,Minimum Qualifications,Preferred Qualifications
0,Google,Google Cloud Program Manager,Program Management,Singapore,"Shape, shepherd, ship, and show technical prog...",BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...
1,Google,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,"Shanghai, China",Drive cross-functional activities in the suppl...,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin..."
2,Google,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"New York, NY, United States",Collect and analyze data to draw insight and i...,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...


In [ ]:
df = df.rename(columns={'Minimum Qualifications': 'Minimum_Qualifications', 'Preferred Qualifications': 'Preferred_Qualifications'})

In [ ]:
df.Company.value_counts()

Google     1227
YouTube      23
Name: Company, dtype: int64

In [ ]:
df.Category.value_counts()

Sales & Account Management          168
Marketing & Communications          165
Finance                             115
Technical Solutions                 101
Business Strategy                    98
People Operations                    86
User Experience & Design             84
Program Management                   74
Partnerships                         60
Product & Customer Support           50
Legal & Government Relations         46
Administrative                       40
Software Engineering                 31
Sales Operations                     31
Hardware Engineering                 26
Real Estate & Workplace Services     25
Manufacturing & Supply Chain         16
Technical Infrastructure             11
Network Engineering                   6
Technical Writing                     5
Developer Relations                   5
IT & Data Management                  5
Data Center & Network                 2
Name: Category, dtype: int64

In [ ]:
df.Location.value_counts()[:10]

Mountain View, CA, United States    190
Sunnyvale, CA, United States        155
Dublin, Ireland                      87
New York, NY, United States          70
London, United Kingdom               62
San Francisco, CA, United States     49
Singapore                            41
Sydney, Australia                    31
Tokyo, Japan                         31
San Bruno, CA, United States         28
Name: Location, dtype: int64

In [ ]:
df['Country'] = df['Location'].apply(lambda x : x.split(',')[-1])
df.Country.value_counts()[:10]

 United States     638
 Ireland            87
 United Kingdom     62
 Germany            54
Singapore           41
 China              38
 Australia          35
 Japan              31
 Taiwan             30
 India              28
Name: Country, dtype: int64

In [ ]:
pd.isnull(df).sum()

Company                      0
Title                        0
Category                     0
Location                     0
Responsibilities            15
Minimum_Qualifications      14
Preferred_Qualifications    14
Country                      0
dtype: int64

In [ ]:
df = df.dropna(how='any',axis='rows')

In [ ]:
pd.isnull(df).sum()

Company                     0
Title                       0
Category                    0
Location                    0
Responsibilities            0
Minimum_Qualifications      0
Preferred_Qualifications    0
Country                     0
dtype: int64

In [ ]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer, MaxAbsScaler
from sklearn.pipeline import make_pipeline


scaler = MaxAbsScaler()

model = NMF(n_components=100)

normalizer = Normalizer()

# Create a pipeline: pipeline
pipeline = make_pipeline(scaler,model,normalizer)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors_Responsibilities = vectorizer.fit_transform(df['Responsibilities'])

In [ ]:
Responsibilities = pipeline.fit_transform(vectors_Responsibilities)

In [ ]:
df_Responsibilities = pd.DataFrame(Responsibilities,index=df['Title'])

In [ ]:
df_Responsibilities.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Google Cloud Program Manager,0.047429,0.163318,0.000000,0.0,0.0,0.075483,0.0,0.000000,0.0,0.0,0.036633,0.0,0.056001,0.090219,0.0,0.0,0.057951,0.0,0.0,0.0,0.0,0.000000,0.131839,0.0,0.0,0.017617,0.0,0.009301,0.0,0.177963,0.0,0.0,0.000000,0.0,0.020372,0.0,0.107564,0.0,0.000000,0.070305,...,0.0,0.0,0.074125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.756943,0.070121,0.0,0.0,0.010483,0.000000,0.0,0.135876,0.107067,0.0,0.000000,0.098255,0.000000,0.050954,0.427439,0.0,0.0,0.018187,0.000000,0.0,0.167415,0.000000,0.000000,0.0,0.0,0.053295,0.0
"Supplier Development Engineer (SDE), Cable/Connector",0.000000,0.000000,0.005621,0.0,0.0,0.082635,0.0,0.026355,0.0,0.0,0.077768,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.042412,0.159649,0.0,0.0,0.022067,0.0,0.000000,0.0,0.000000,0.0,0.0,0.033665,0.0,0.000000,0.0,0.005994,0.0,0.722553,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000183,0.134832,0.0,0.0,0.000000,0.122753,0.0,0.000000,0.008045,0.0,0.025626,0.232672,0.030077,0.007590,0.000000,0.0,0.0,0.285496,0.006333,0.0,0.000000,0.485602,0.065806,0.0,0.0,0.000000,0.0


In [ ]:
pd.set_option('display.max_colwidth', -1)
print(df[df.Title.str.contains('Data Scientist')]['Title'])

277    Customer Experience Data Scientist, Google Cloud Support         
875    Lead Data Scientist                                              
909    Data Scientist / Quantitative Analyst Technical Lead, Engineering
933    Data Scientist / Quantitative Analyst Technical Lead, Engineering
Name: Title, dtype: object


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
Position = df_Responsibilities.loc['Customer Experience Data Scientist, Google Cloud Support']

In [ ]:
similarities_1 = df_Responsibilities.dot(Position)

In [ ]:
similarities_1[:3]

Title
Google Cloud Program Manager                                             0.421618
Supplier Development Engineer (SDE), Cable/Connector                     0.044176
Data Analyst, Product and Tools Operations, Google Technical Services    0.074167
dtype: float64

In [ ]:
print(similarities_1.nlargest())

Title
Customer Experience Data Scientist, Google Cloud Support                1.000000
Internal Executive Communications and Event Producer                    0.591827
Marketing Analytics Manager, Retail Marketing                           0.590334
Business Analyst, Google Play Partner Promotions Strategy               0.553235
Legal and Policy Operations Specialist (English and Korean), YouTube    0.552712
dtype: float64


In [ ]:
df[np.isin(df['Title'],similarities_1.nlargest().index.tolist())].head()

,Company,Title,Category,Location,Responsibilities,Minimum_Qualifications,Preferred_Qualifications,Country
277,Google,"Customer Experience Data Scientist, Google Cloud Support",Program Management,"New York, NY, United States","Create, design and maintain reusable datasets with appropriate architecture to deal with high level of data input complexity.\nProvide analytical decision making support for key projects when it comes to providing customer experience (CX) insights. Research new ways of modeling data proactively to unlock actionable customer insights or improve processes.\nContribute to the development of the analytical frameworks, tools, and skills for the Google Cloud Support team.\nWork with internal teams to translate data and model results into tactical and strategic insights that are clear, complete, accurate and relevant. Be able to provide insights that are understandable and applicable to decision making and meets the needs of varying client audiences; research new algorithms and methods for optimizing quality and support experience improvements.\nPartner with cross-functional Business Intelligence teams, as well as non-technical stakeholders, to help frame problems, document and prioritize data science requirements. Communicate results of complex technical work clearly.","MBA/MS/MA degree with a quantitative focus (economics, computer science, mathematics, physics, engineering, statistics, or accounting)\n5 years of quantitative analytics experience with a focus on data analysis, statistical modeling, machine learning, and/or predictive analytics.\nExperience with statistical software (e.g., R, S-Plus, SAS, or similar).\nExperience with databases and scripting languages (e.g., Python).","PhD degree with a quantitative focus (economics, computer science, mathematics, physics, engineering, statistics, or accounting)\nProven experience delivering bespoke analytics to senior stakeholders (problem scoping/definition, modeling, interpretation, presentation)\nExperience in a multi-channel client/customer support environment (e.g., email, phone, chat, social).\nAbility to consistently deliver quality outputs in a high paced and dynamic environment, work through ambiguity, with an ability to work to tight deadlines.\nDemonstrated ability to build strong relationships at all levels as a cross-functional influencer.\nStrong oral and written communication skills, including the ability to communicate complex findings in a structured and clear manner to technical and non-technical audiences.",United States
642,Google,Internal Executive Communications and Event Producer,Marketing & Communications,"Sunnyvale, CA, United States","Lead the creative development and delivery of key internal meetings for Google Cloud.\nManage projects working across multiple concurrent deadlines, keeping track of budgets, empowering team members and hiring freelance support if needed.\nWork effectively with global teammates and executives (sometimes across multiple locations and time-zones) to gather feedback and input, ensuring stakeholders are informed at all times.\nAssist with event logistics when necessary.","BA/BS degree or equivalent practical experience.\n10 years of experience in internal communications or executive communications.\nExperience managing event timetables, budgets and production staff (in house or contracted).","Experience with script writing.\nExperience managing agencies and employees for an in-house creative team and/or serving as a client manager at an agency.\nProficiency in presentation software, video production, and live meeting production.\nDemonstrated ability to work calmly and maintain good judgment in fast-paced, dynamic environments.\nAbility to interact with executives.\nExcellent written and verbal communications skills.",United States
740,Google,"Marketing Analytics Manager, Retail Marketing",Marketing & Communications,"Mountain View, CA, United States","Analyze sales and brand uplift due to our retail marketin

In [ ]:
type(similarities_1)

pandas.core.series.Series

In [ ]:
vectorizer_Requirements = TfidfVectorizer()
vectors_Requirements = vectorizer_Requirements.fit_transform(df['Minimum_Qualifications'])


In [ ]:
Requirements = pipeline.fit_transform(vectors_Requirements)


In [ ]:
df_Requirementss = pd.DataFrame(Requirements,index=df['Title'])

In [ ]:
Position = df_Requirementss.loc['Customer Experience Data Scientist, Google Cloud Support']

In [ ]:
similarities_2 = df_Responsibilities.dot(Position)

In [ ]:
print(similarities_2.nlargest())

Title
Internal Audit, Technology Auditor                         0.645370
Finance Auditor, Internal Audit                            0.638330
Senior Technology Auditor, Internal Audit                  0.638121
Internal Controls (SOX) Manager, Cloud Business Finance    0.634723
Internal Audit, Infrastructure Technology Audit Manager    0.633784
dtype: float64


In [ ]:
similarities_1

Title
Google Cloud Program Manager                                             0.421618
Supplier Development Engineer (SDE), Cable/Connector                     0.044176
Data Analyst, Product and Tools Operations, Google Technical Services    0.074167
Developer Advocate, Partner Engineering                                  0.000292
Program Manager, Audio Visual (AV) Deployments                           0.194780
                                                                           ...   
Global Investigator                                                      0.187873
Campus Security Manager                                                  0.239942
Facilities Manager                                                       0.143155
Physical Security Manager                                                0.004818
Physical Security Manager                                                0.000000
Length: 1235, dtype: float64

In [ ]:
similarities_1.rename("similarity")
similarities_2.rename("similarity")

similarities_1.to_frame().join(similarities_2.to_frame(),lsuffix='1')


,01,0
Title,,
"APAC Accounting Lead, Non-Ads Products",0.024931,0.044352
"APAC Manager, Employment Tax",0.157902,0.095959
"APAC SMB Account Manager, Google Cloud (Mandarin, Cantonese)",0.141556,0.123774
AR and VR Writer,0.123728,0.044931
ASIC Physical Design Engineering Tech Lead & Manager,0.153088,0.168369
...,...,...
"Web Solutions Engineer, Product and Tools Operations",0.073588,0.175571
"Web Solutions Engineer, Product and Tools Operations",0.073588,0.175571
"Web Solutions Engineer, Product and Tools Operations",0.073588,0.175571


In [ ]:
similarities_overall = (2 * similarities_1) + similarities_2

In [ ]:
print(similarities_overall.nlargest())

Title
Customer Experience Data Scientist, Google Cloud Support     2.179504
Marketing Analytics Manager, Retail Marketing                1.429844
Compensation Analyst                                         1.257592
Internal Executive Communications and Event Producer         1.198581
Business Analyst, Google Play Partner Promotions Strategy    1.196919
dtype: float64


In [ ]:
df[np.isin(df['Title'],similarities_overall.nlargest(3).index.tolist())].head()

,Company,Title,Category,Location,Responsibilities,Minimum_Qualifications,Preferred_Qualifications,Country
277,Google,"Customer Experience Data Scientist, Google Cloud Support",Program Management,"New York, NY, United States","Create, design and maintain reusable datasets with appropriate architecture to deal with high level of data input complexity.\nProvide analytical decision making support for key projects when it comes to providing customer experience (CX) insights. Research new ways of modeling data proactively to unlock actionable customer insights or improve processes.\nContribute to the development of the analytical frameworks, tools, and skills for the Google Cloud Support team.\nWork with internal teams to translate data and model results into tactical and strategic insights that are clear, complete, accurate and relevant. Be able to provide insights that are understandable and applicable to decision making and meets the needs of varying client audiences; research new algorithms and methods for optimizing quality and support experience improvements.\nPartner with cross-functional Business Intelligence teams, as well as non-technical stakeholders, to help frame problems, document and prioritize data science requirements. Communicate results of complex technical work clearly.","MBA/MS/MA degree with a quantitative focus (economics, computer science, mathematics, physics, engineering, statistics, or accounting)\n5 years of quantitative analytics experience with a focus on data analysis, statistical modeling, machine learning, and/or predictive analytics.\nExperience with statistical software (e.g., R, S-Plus, SAS, or similar).\nExperience with databases and scripting languages (e.g., Python).","PhD degree with a quantitative focus (economics, computer science, mathematics, physics, engineering, statistics, or accounting)\nProven experience delivering bespoke analytics to senior stakeholders (problem scoping/definition, modeling, interpretation, presentation)\nExperience in a multi-channel client/customer support environment (e.g., email, phone, chat, social).\nAbility to consistently deliver quality outputs in a high paced and dynamic environment, work through ambiguity, with an ability to work to tight deadlines.\nDemonstrated ability to build strong relationships at all levels as a cross-functional influencer.\nStrong oral and written communication skills, including the ability to communicate complex findings in a structured and clear manner to technical and non-technical audiences.",United States
740,Google,"Marketing Analytics Manager, Retail Marketing",Marketing & Communications,"Mountain View, CA, United States","Analyze sales and brand uplift due to our retail marketing executions, by analyzing a wide variety of data.\nLead global analyses of Retail Sales Associate training and engagement impact. Work with internal Retail Operations team to gather appropriate data via store collection or surveys and build models to estimate ROI of various training programs and help influence future business development.\nLead global analysis of in-store demo device analytics. Work with internal Platform team and external developer of demo mode to gather data and analyze by country.\nLead development and analysis of datasets to measure retail reach globally at the store level; tie to other data sets in order to run statistical analyses with sales, brand impact, and other measures of success. Develop compelling data visualizations and dashboards to communicate insights effectively.\nWork cross-functionally with data analyst teams across the hardware and marketing organizations to build data tools (data warehouses, dashboards etc) and drive alignment and ensure a single version of validity between data sources.","Bachelor’s degree in Data Science, Statistics, Engineering, Quantitative Finance or other analytics- related field or equivalent practical experience.\n5 years of business experience.\nIndustry experience in SQL, as well as

In [ ]:
from scipy.cluster.vq import kmeans, vq
from numpy import random

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'